In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/'
data_folder = parent_folder + 'data/'
from hydra import compose, initialize
with initialize(config_path="../../conf"):
    # Compose the configuration
    cfg = compose(config_name="TopicClusterMapping.yaml")

import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_41620/4032682437.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="../../conf"):


In [1]:
import requests

In [2]:
import sys

sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian/')

from src.clustering.ClustersResetService import ClustersResetService
from src.clustering.ClusterAssignmentService import ClusterAssignmentService
from sql.clustering.ClusteringSQL import ClusteringSQL
from src.onboarding.TopicClusterMapping import TopicClusterMapping

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-pac

In [3]:
from sql.onboarding.TopicPreferencesSQL import TopicPreferencesSQL

In [4]:
# TopicPreferencesSQL.reset_all_topic_preferences_in_db()

In [4]:
tc = TopicClusterMapping(cfg=cfg)

In [5]:
tc.recompute_preferences_for_all_topics()

In [5]:
%timeit tc.get_merged_cluster_preferences_for_topic_list(topic_id_list=[1,2,3])

121 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
for topic in [1,3,4,5,6,7,8,9]:
    tc.compute_and_save_cluster_priors_for_topic_id(topic_id=topic)

In [8]:
%timeit TopicPreferencesSQL.get_all_topic_preferences_from_db()

86 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
ap = TopicPreferencesSQL.get_all_topic_preferences_from_db()

In [14]:
pref_dict = {}
for row in ap[ap.topic_id == 2][['cluster_id','a','b']].itertuples():
    pref_dict[row.cluster_id] = {'a': float(row.a), 'b': float(row.b)}

In [16]:
sample_topic_list = [1,3,5]

In [19]:
ap[ap.topic_id == 49]

,topic_id,topic_name,cluster_id,a,b,clustering_run_id,created_at


In [8]:
def merge_dicts(dict_list):
    merged_dict = {}
    
    for d in dict_list:
        for key, value in d.items():
            if key not in merged_dict:
                merged_dict[key] = value
            else:
                # Merge 'a' values using the given logic: 1 + (a1-1) + (a2-1)
                merged_dict[key]['a'] = min(5.0, 1 + (merged_dict[key]['a'] - 1) + (value['a'] - 1))
                
                # Merge 'b' values by taking the average
                merged_dict[key]['b'] = (merged_dict[key]['b'] + value['b']) / 2
                
    return merged_dict

In [10]:
# Given list of strings
categories = [
    "Investing Basics",
    "Personal Finance",
    "Taxation",
    "Retirement Goals",
    "Insurance",
    "Credit Health",
    "Real Estate Guide",
    "Tech & Startups",
    "Industry Highlights",
    "Green Investing",
    "Study & Education Loans",
    "Trade & Global Markets",
    "Travel & Lifestyle",
    "Precious Metals & Commodities",
    "Small Business Strategies",
    "Banking & New Offers",
    "Government Benefits",
    "Forex & Currency",
    "IPOs, M&As",
    "Global Economy"
]

# Convert the list to a dictionary with keys as numbers and values as strings
category_dict = {i: category for i, category in enumerate(categories, 1)}

# Print the resulting dictionary
print(category_dict)


{1: 'Investing Basics', 2: 'Personal Finance', 3: 'Taxation', 4: 'Retirement Goals', 5: 'Insurance', 6: 'Credit Health', 7: 'Real Estate Guide', 8: 'Tech & Startups', 9: 'Industry Highlights', 10: 'Green Investing', 11: 'Study & Education Loans', 12: 'Trade & Global Markets', 13: 'Travel & Lifestyle', 14: 'Precious Metals & Commodities', 15: 'Small Business Strategies', 16: 'Banking & New Offers', 17: 'Government Benefits', 18: 'Forex & Currency', 19: 'IPOs, M&As', 20: 'Global Economy'}


In [11]:
k = merge_dicts([dct, dct])

In [14]:
import pandas as pd

In [20]:
df = pd.DataFrame(k).T

In [21]:
df.empty

False

In [5]:
dct = tc.compute_cluster_priors_for_keyword('sensex')

In [5]:
cpd = tc.compute_cluster_preferences(['sensex'])

In [6]:
tc.position_log_scaling

{0: 0.430676558073393,
 1: 0.6826061944859853,
 2: 0.861353116146786,
 3: 1.0,
 4: 1.1132827525593783,
 5: 1.2090619551221675,
 6: 1.292029674220179,
 7: 1.3652123889719705,
 8: 1.430676558073393,
 9: 1.4898961024049782,
 10: 1.5439593106327714,
 11: 1.5936926411670822,
 12: 1.6397385131955604,
 13: 1.6826061944859854,
 14: 1.722706232293572,
 15: 1.7603744277225877,
 16: 1.7958889470453634,
 17: 1.8294828004351502,
 18: 1.861353116146786,
 19: 1.8916681496081527,
 20: 1.9205726604783708,
 21: 1.9481920934663794,
 22: 1.9746358687061643,
 23: 2.0,
 24: 2.0243691992404753,
 25: 2.0478185834579556,
 26: 2.0704150712689535,
 27: 2.0922185341674995,
 28: 2.1132827525593783,
 29: 2.1336562149773224,
 30: 2.1533827903669653,
 31: 2.172502296890963,
 32: 2.191050985795981,
 33: 2.2090619551221673,
 34: 2.2265655051187565,
 35: 2.2435894449529292,
 36: 2.260159358508543,
 37: 2.2762988356530673,
 38: 2.2920296742201787,
 39: 2.307372057048123,
 40: 2.322344707681546,
 41: 2.3369650277500638,
 

In [7]:
cpd._data

{512: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 515: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 519: {'ts_clicked': 4.925, 'ts_not_clicked': 3.0},
 392: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 526: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 528: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 530: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 531: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 537: {'ts_clicked': 1.645, 'ts_not_clicked': 3.0},
 540: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 289: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 421: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 293: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 297: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 312: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 472: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 484: {'ts_clicked': 2.025, 'ts_not_clicked': 3.0},
 488: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 493: {'ts_clicked': 1.0, 'ts_not_clicked': 3.0},
 503: {'ts_clicked': 1.0, 'ts_not_clicked': 

In [6]:
from src.feed._utils import get_time_decay_article_priors_for_all_interacted_articles

In [7]:
df = get_time_decay_article_priors_for_all_interacted_articles()

In [8]:
df

,article_id,a_all,b_all,ah,bh
0,6512cde8395f46458f8bb717,1,2,0.004722,0.004740
1,6512cde9395f46458f8bb718,2,2,0.000999,0.000999
2,6512cdef395f46458f8bb71b,2,3,0.017434,0.020456
3,6512cdf0395f46458f8bb71d,2,4,0.000087,0.004894
4,6512cdf2395f46458f8bb71f,5,7,0.102971,0.138415
...,...,...,...,...,...
8230,6513aa31395f46458f8c0281,1,4,0.000005,0.002808
8231,6513aa35395f46458f8c0282,0,2,0.000000,0.000012
8232,6513aa3a395f46458f8c0283,3,3,0.011544,0.011544
8233,6513aa3c395f46458f8c0284,5,8,0.159355,0.177657


In [9]:
from sql.feed.ArticleSelectionSQL import ArticleSelectionSQL

In [13]:
import pandas as pd

In [10]:
cdf = ArticleSelectionSQL.get_candidate_articles()

In [15]:
cdf['a'] = 1
cdf['b'] = 8

In [20]:
df1 = pd.DataFrame()

In [21]:
cdf = pd.merge(cdf, df1, how = 'left', on = 'article_id')

KeyError: 'article_id'

In [18]:
cdf['a'] = cdf['a'] + cdf['ah']

In [19]:
cdf

,article_id,cluster_id,a,b,a_all,b_all,ah,bh
0,651e1e70a662d76276b89434,201,NaN,8,NaN,NaN,NaN,NaN
1,651ddbd5a662d76276b7eff5,191,NaN,8,NaN,NaN,NaN,NaN
2,651e0c7ba662d76276b86a27,191,NaN,8,NaN,NaN,NaN,NaN
3,651de906a662d76276b81302,203,NaN,8,NaN,NaN,NaN,NaN
4,651e0751a662d76276b85e52,201,NaN,8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
22821,651df9aba662d76276b83c50,185,NaN,8,NaN,NaN,NaN,NaN
22822,651df3eaa662d76276b82f88,201,NaN,8,NaN,NaN,NaN,NaN
22823,651dd2fda662d76276b7d9a3,205,NaN,8,NaN,NaN,NaN,NaN
22824,651e03c8a662d76276b855a7,201,NaN,8,NaN,NaN,NaN,NaN
